# Functional Programming SS19

# Assignment 06 Solutions

### Exercise 1 (Fixpoints)

*Give the value of $Val[\![ e ]\!]_{\rho}$ for the following simple Haskell expressions $e \in \{ e_1, e_2, e_3 \}$ for the environment $\rho = \omega + \rho'$, where $\omega$ is the initial environment and $\rho'$ is he environment with $\rho'(x) = 2, \rho'(y) = 18, \rho'(z) = 3$, and $\rho'$ is undefined for all other variables.*

*Describe your computation in detail by applying rules for $Val$ step by step. Also, for each higher-order function $f : \text{Dom} \rightarrow \text{Dom}$, where $\operatorname{lfp} f$ is needed in the calculation, determine what the function $f^{n}(\bot), n \in \mathbb{N}$, computes.*

```
e1 = let isAnswer = \x -> if x < 17 then 2 else 3
     in isAnswer y

e2 = let sum = \x -> if x < 4 then x + sum (x+1)
                     else 0
     in sum x

e3 = let pow = \y -> \ z -> if z <= 0 then 1
                            else y * pow y (z-1)
     in pow 4 z
```


**Hints**:
* You may switch between infix- and prefix-notation for Haskell operators without any intermediate steps if needed.
* You may simplify $Val[\![ exp_1 \; exp_2 \; exp_3 ]\!]_{\rho}$ to $f \; (Val[\![ exp_2 ]\!]_{\rho}) \; (Val[\![ exp_3 ]\!]_{\rho})$ where $f = Val[\![ exp_1 ]\!]_{\rho}$ *in* Functions *in* Dom in one step if $exp_1$ represents a function expecting two arguments, e.g. $+, -, *, <, <=$, etc.

* `e1 = let isAnswer = \x -> if x < 17 then 2 else 3 in isAnswer y`

\begin{align*}
    Val[\![ e1 ]\!] &= Val[\![ \text{let isAnswer = $\backslash$x -> if x < 17 then 2 else 3 in isAnswer y} ]\!]_{\rho} \\
    &= Val[\![ \text{isAnswer y} ]\!](\rho + \{ \text{isAnswer} / \operatorname{lfp} \; f \})
\end{align*}

for $f$ *in* Functions *in* Dom where

\begin{equation*}
    f(k) = Val[\![ \text{$\backslash$x -> if x < 17 then 2 else 3} ]\!](\rho + \{ \text{isAnswer} / k \} )
\end{equation*}

Here, we have $Val[\![ \text{$\backslash$x -> if x < 17 then 2 else 3} ]\!] = h$ *in* Functions *in* Dom where

\begin{align*}
    h(d) &= Val[\![ \text{if x < 17 then 2 else 3} ]\!](\rho + \{ x / d \}) \\
    &= \left\{
        \begin{array}{ll}
            2 & \text{if } Val[\![ \text{<(x 17)} ]\!](\rho + \{ x / d \}) = \text{True in Constructions}_0\text{ in Dom} \\
            3 & \text{if } Val[\![ \text{<(x 17)} ]\!](\rho + \{ x / d \}) = \text{False in Constructions}_0\text{ in Dom} \\
            \bot & \text{otherwise}
        \end{array}
    \right.
\end{align*}

such that $Val[\![ \text{<(x 17)} ]\!](\rho + \{ x \; / \; d \}) = g$ *in* Functions *in* Dom.

Since isAnswer is not a recursive function, we have that $\operatorname{lfp} f = h$ and so

\begin{align*}
    Val[\![ \text{e1} ]\!] &= Val[\![ \text{isAnswer y} ]\!](\rho + \{ \text{isAnswer} \; / \; h ) \\
    &= Val[\![ \text{if x < 17 then 2 else 3} ]\!] (\rho + \{ x \; / \; y \}) \\
    &= Val[\![ \text{if x < 17 then 2 else 3} ]\!] (\{ x \; / \; 18 \}) \\
    &= Val[\![ \text{if 18 < 17 then 2 else 3} ]\!] \\
    &= 3
\end{align*}

* `e2 = let sum = \x -> if x < 4 then x + sum (x+1) else 0 in sum x`

\begin{align*}
    Val[\![ e2 ]\!] &= Val[\![ \text{let sum = $\backslash$x -> if x < 4 then x + sum (x+1) else 0 in sum x} ]\!]_{\rho} \\
    &= Val[\![ \text{sum x} ]\!](\rho + \{ \text{sum} / \operatorname{lfp} \; f \})
\end{align*}

for $f$ *in* Functions *in* Dom where

\begin{equation*}
    f(k) = Val[\![ \text{$\backslash$x -> if x < 4 then x + sum (x+1) else 0} ]\!](\rho + \{ \text{sum} / k \} )
\end{equation*}

As above, we have $Val[\![ \text{$\backslash$x -> if x < 4 then x + sum (x+1) else 0} ]\!] = h$ *in* Functions *in* Dom where

\begin{align*}
    h(d) &= Val[\![ \text{if x < 4 then x + sum (x+1) else 0} ]\!](\rho + \{ \text{sum} / \operatorname{lfp} \; f, \; x / d \})\\
    &= \left\{
        \begin{array}{ll}
            \text{x + sum(x+1)} & \text{if } Val[\![ \text{<(x, 4)} ]\!](\rho + \{ \text{sum} / \operatorname{lfp} \; f, \; x / d \}) = \text{True in Constructions}_0\text{ in Dom} \\
            0 & \text{if } Val[\![ \text{<(x, 4)} ]\!](\rho + \{ \text{sum} / \operatorname{lfp} \; f, \; x / d \}) = \text{False in Constructions}_0\text{ in Dom} \\
            \bot & \text{otherwise}
        \end{array}
    \right.
\end{align*}

such that $Val[\![ \text{<(x 4)} ]\!](\rho + \{ \text{sum} / \operatorname{lfp} \; f, \; x \; / \; d \}) = g$ *in* Functions *in* Dom.

We are now dealing with a recursive function, so let us consider $f^n(\bot), n \in \mathbb{N}$:

\begin{align*}
    g_0 &= f^0(\bot) = \bot \\
    g_1 &= f^1(\bot) = 
    \left\{
        \begin{array}{ll}
            x + \bot(x+1) & \text{if } x < 4 \\
            0 & \text{if } x \geq 4 \\
            \bot & \text{otherwise}
        \end{array}
    \right. = \left\{
        \begin{array}{ll}
            \bot & \text{if } x < 4 \\
            0 & \text{if } x \geq 4 \\
            \bot & \text{otherwise}
        \end{array}
    \right. \\
    g_2 &= f^2(\bot) = f(g_1) =
    \left\{
        \begin{array}{ll}
            x + g_1(x+1) & \text{if } x < 4 \\
            0 & \text{if } x \geq 4 \\
            \bot & \text{otherwise}
        \end{array}
    \right. = \left\{
        \begin{array}{ll}
            \bot & \text{if } x < 3 \\
            3 & \text{if } x = 3 \\
            0 & \text{if } x > 4 \\
            \bot & \text{otherwise}
        \end{array}
    \right. \\
    g_3 &= f^3(\bot) = f(g_2) =
    \left\{
        \begin{array}{ll}
            x + g_2(x+1) & \text{if } x < 4 \\
            0 & \text{if } x \geq 4 \\
            \bot & \text{otherwise}
        \end{array}
    \right. = \left\{
        \begin{array}{ll}
            \bot & \text{if } x < 3 \\
            5 & \text{if } x = 2 \\
            3 & \text{if } x = 3 \\
            0 & \text{if } x > 4 \\
            \bot & \text{otherwise}
        \end{array}
    \right.
\end{align*}

This is thus the function that computes the sum of all number from $x$ up to $3$ and returns $0$ otherwise; thus

\begin{align*}
    \operatorname{lfp} \; f = \left\{
        \begin{array}{ll}
            \sum_{i=x}^{3}i & \text{if } x < 4 \\
            0 & \text{if } x \geq 4 \\
            \bot & \text{otherwise}
        \end{array}
    \right.
\end{align*}

We thus have that

\begin{align*}
    Val[\![ \text{e2} ]\!] &= Val[\![ \text{sum x} ]\!](\rho + \{ \text{sum} \; / \; \operatorname{lfp} \; f ) \\
    &= Val[\![ \operatorname{lfp} \; f \; \text{x} ]\!] (\rho) \\
    &= Val[\![ \operatorname{lfp} \; f \; \text{2} ]\!] \\
    &= \sum_{i=2}^{3}i = 5
\end{align*}

* `e3 = let pow = \y -> \ z -> if z <= 0 then 1 else y * pow y (z-1) in pow 4 z`

\begin{align*}
    Val[\![ e3 ]\!] &= Val[\![ \text{let pow = $\backslash$y -> $\backslash$z -> if z <= 0 then 1 else y * pow y (z-1) in pow 4 z} ]\!]_{\rho} \\
    &= Val[\![ \text{pow 4 z} ]\!](\rho + \{ \text{pow} / \operatorname{lfp} \; f \})
\end{align*}

for $f$ *in* Functions *in* Dom where

\begin{equation*}
    f(k) = Val[\![ \text{$\backslash$y -> $\backslash$z -> if z <= 0 then 1 else y * pow y (z-1) in pow 4 z} ]\!](\rho + \{ \text{pow} / k \} )
\end{equation*}

We have $Val[\![ \text{$\backslash$y -> $\backslash$z -> if z <= 0 then 1 else y * pow y (z-1)} ]\!] = h$ *in* Functions *in* Dom where

\begin{equation*}
    h(d_1) = Val[\![ \text{$\backslash$z -> if z <= 0 then 1 else y * pow y (z-1)} ]\!](\rho + \{ \text{pow} / \operatorname{lfp} \; f, \; y / d_1 \})\\
\end{equation*}

Since we have another lambda expression, we find that $Val[\![ \text{$\backslash$z -> if z <= 0 then 1 else y * pow y (z-1)} ]\!] = l$ *in* Functions *in* Dom and

\begin{align*}
    l(d_2) &= Val[\![ \text{if z <= 0 then 1 else y * pow y (z-1)} ]\!](\rho + \{ \text{pow} / \operatorname{lfp} \; f, \; y / d_1, \; z / d_2 \})\\
    &= \left\{
        \begin{array}{ll}
            \text{1} & \text{if } Val[\![ \text{<=(z, 0)} ]\!](\rho + \{ \text{pow} / \operatorname{lfp} \; f, \; y / d_1, \; z / d_2 \}) = \text{True in Constructions}_0\text{ in Dom} \\
            \text{y * pow y (z-1)} & \text{if } Val[\![ \text{<=(z, 0)} ]\!](\rho + \{ \text{pow} / \operatorname{lfp} \; f, \; y / d_1, \; z / d_2 \}) = \text{False in Constructions}_0\text{ in Dom} \\
            \bot & \text{otherwise}
        \end{array}
    \right.
\end{align*}

such that $Val[\![ \text{<=(z 0)} ]\!](\rho + \{ \text{pow} / \operatorname{lfp} \; f, \; y / d_1, \; z / d_2 \}) = g$ *in* Functions *in* Dom.

Let us now consider $f^n(\bot), n \in \mathbb{N}$:

\begin{align*}
    g_0 &= f^0(\bot) = \bot \\
    g_1 &= f^1(\bot) = 
    \left\{
        \begin{array}{ll}
            1 & \text{if } z \leq 0 \\
            y * \bot(z-1) & \text{if } z > 0 \\
            \bot & \text{otherwise}
        \end{array}
    \right. = \left\{
        \begin{array}{ll}
            1 & \text{if } z \leq 0 \\
            \bot & \text{if } z > 0 \\
            \bot & \text{otherwise}
        \end{array}
    \right. \\
    g_2 &= f^2(\bot) = f(g_1) =
    \left\{
        \begin{array}{ll}
            1 & \text{if } z \leq 0 \\
            y * g_1(z-1) & \text{if } z > 0 \\
            \bot & \text{otherwise}
        \end{array}
    \right. = \left\{
        \begin{array}{ll}
            1 & \text{if } z \leq 0 \\
            y & \text{if } z = 1 \\
            \bot & \text{if } z > 1 \\
            \bot & \text{otherwise}
        \end{array}
    \right. \\
    g_3 &= f^3(\bot) = f(g_2) =
    \left\{
        \begin{array}{ll}
            1 & \text{if } z \leq 0 \\
            y * g_2(z-1) & \text{if } z > 0 \\
            \bot & \text{otherwise}
        \end{array}
    \right. = \left\{
        \begin{array}{ll}
            1 & \text{if } z \leq 0 \\
            y & \text{if } z = 1 \\
            y^2 & \text{if } z = 2 \\
            \bot & \text{if } z > 2 \\
            \bot & \text{otherwise}
        \end{array}
    \right.
\end{align*}

which is the function that computes the power $z$-th of a positive integer $y$; thus

\begin{align*}
    \operatorname{lfp} \; f(x) = \left\{
        \begin{array}{ll}
            y^z & \text{if } z \leq 0 \\
            \bot & \text{otherwise}
        \end{array}
    \right.
\end{align*}

We thus have that

\begin{align*}
    Val[\![ \text{e3} ]\!] &= Val[\![ \text{pow 4 z} ]\!](\rho + \{ \text{pow} / \operatorname{lfp} \; f \}) \\
    &= Val[\![ \operatorname{lfp} \; f \; \text{4 z} ]\!] (\rho) \\
    &= Val[\![ \operatorname{lfp} \; f \; \text{4 3} ]\!] \\
    &= 4^3 = 64
\end{align*}